# Context-Aware Object Insertion Using Stable Diffusion and Vision-Language Models

**This Colab notebook is a collaborative effort by:**

* Dekshita Sriyaa K.
* Anagha S Bharadwaj
* Aditi S Kulkarni
* Akshat Singh Jaswal


#Installing dependencies
This code sets up the environment for our project by installing essential libraries:
* **transformers:** This library is  used for working with pre-trained models .
* **diffusers:** This library is used for working with diffusion models, a type of generative model used for image generation. In our case, it's used for Stable Diffusion.
* **bitsandbytes:** This library is for optimizing models by reducing their precision . This can improve performance on some hardware.
* **accelerate:** This library is for accelerating training or inference of models, especially on multiple GPUs or TPUs.
* **gradio:** This library is  used for creating web interfaces for  machine learning models.


In [ ]:
!pip install transformers diffusers
!pip install bitsandbytes
!pip install accelerate
!pip install gradio

# Importing all essential libraries
This code block imports libraries for various functionalities in our project:

* **transformers:** This library likely provides functions for working with pre-trained language models.
    * `AutoProcessor`: This class simplifies preprocessing text data for compatibility with specific models.
    * `LlavaForConditionalGeneration`: This class is a pre-trained model for image-to-text generation tasks.
    * `BitsAndBytesConfig`: This class is used for configuring model quantization with bitsandbytes.
* **diffusers:** This library provides tools for diffusion models.
    * `StableDiffusionInpaintPipeline`: This class is used for image inpainting with Stable Diffusion, allowing insertion of objects into existing images.
* **torch:** This library provides tools for deep learning operations, used for manipulating tensors.
* **PIL (Python Imaging Library):** This library provides tools for image processing.
    * `Image`, `ImageDraw`, `ImageFont`: These classes are likely used for image loading, drawing, and adding text elements.
* **matplotlib.pyplot:** This library is used for plotting and visualizing images.
* **gradio:** This library is used for creating web interfaces for the project.
* **io:** This library provides tools for working with input/output streams, used for handling image data.

In [ ]:
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
from diffusers import StableDiffusionInpaintPipeline
import torch
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import gradio as gr
import io

# Prompt Generation using the Llava model with quantization

This code defines a configuration object (`quantization_config`) for model quantization using the `BitsAndBytesConfig` class.
This part of the code defines `"llava-hf/llava-1.5-7b-hf"` and loads the pre-trained Llava model (`LlavaForConditionalGeneration`) from the transformers library. It also applies the defined `quantization_config` for model size reduction.

In [ ]:
# Step 1: Generate prompt using Llava model
#!pip install accelerate
# Load Llava model and processor with quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,  # Enable double quantization for further memory optimization
    load_in_8bit_fp32_cpu_offload=True  # Offload parts of the model to CPU
)
model_id1 = "llava-hf/llava-1.5-7b-hf"
processor = AutoProcessor.from_pretrained(model_id1)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id1,
    quantization_config=quantization_config,
    device_map="auto"  # Let Transformers automatically decide the device mapping
)


Unused kwargs: ['load_in_8bit_fp32_cpu_offload']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/models/llava/configuration_llava.py:100: FutureWarning: The `vocab_size` argument is deprecated and will be removed in v4.42, since it can be inferred from the `text_config`. Passing this argument has no effect
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

**This part of the code logins into hugging face hub to be able to access the models**

In [ ]:
import os
from PIL import Image
import cv2
import numpy as np
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionInpaintPipeline
from huggingface_hub import notebook_login
import matplotlib.pyplot as plti

device = 'cuda'

# Log in to Hugging Face
notebook_login()

#Loading Stable Diffusion pipelines for image inpainting.
 This code block loads a separate Stable Diffusion Inpainting pipeline from the Hugging Face Hub using the `StableDiffusionInpaintPipeline.from_pretrained` function. This pipeline is specifically designed for inpainting tasks, which involve filling in missing parts of an image while maintaining coherence with the existing content.

In [ ]:
# Load the Stable Diffusion pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    'CompVis/stable-diffusion-v1-4', revision='fp16',
    torch_dtype=torch.float16, use_auth_token=True
).to(device)

# Load the inpainting pipeline
inpaint_pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16
).to(device)

/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_loading_utils.py:219: FutureWarning: You are loading the variant fp16 from CompVis/stable-diffusion-v1-4 via `revision='fp16'` even though you can load it via `variant=`fp16`. Loading model variants via `revision='fp16'` is deprecated and will be removed in diffusers v1. Please use `variant='fp16'` instead.
  warnings.warn(
vae/diffusion_pytorch_model.safetensors not found
Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--CompVis--stable-diffusion-v1-4/snapshots/2880f2ca379f41b0226444936bb7a6766a227587/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--CompVis-

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/51388a731f57604945fddd703ecb5c50e8e7b49d/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


**In essence, the create_mask function takes an image and a rectangular area as input and generates a binary mask image. The white rectangle on the mask defines the region where the inpainting pipeline will focus on filling in missing information based on the surrounding context and the text prompt.**

In [ ]:
# Function to create a mask for the inpainting
def create_mask(image, mask_area):
    mask = Image.new("L", image.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.rectangle(mask_area, fill=255)
    return mask

# Inpainting object into scene
This code defines a function `combine_image_and_mask` for combining an image and its mask into a single tensor suitable for inpainting.
- The function takes an original image and its corresponding inpainting mask as inputs (both as PIL Image objects).
- It converts them to a suitable format for the Stable Diffusion inpainting pipeline
- This combined tensor includes both the image information and the mask defining the region where inpainting should occur.
- By providing this combined tensor to the inpainting pipeline, the function simplifies the process of feeding data and allows the model to focus on filling in the designated area while considering the surrounding image context.


In [ ]:
def generate_combined_image(scene_img, object_img):
    try:
        # Generate prompt using the object image
        prompts = [
            "USER: <image>\nWrite me a prompt describing the object in the focus of this image and asking it to be put in the image with similar perspective and size.\nASSISTANT:"
        ]
        inputs = processor(prompts, images=[object_img], padding=True, return_tensors="pt").to("cuda")
        output = model.generate(**inputs, max_new_tokens=45)
        generated_text = processor.batch_decode(output, skip_special_tokens=True)
        prompt = generated_text[0].split("ASSISTANT:")[-1].strip()

        #print(f"Generated Prompt: {prompt}")

        # Define the mask area (you might want to customize this)
        mask_area = (150, 150, 400, 400)

        # Create the mask
        mask = create_mask(scene_img, mask_area)

        # Perform inpainting
        generated_image = inpaint_pipe(prompt=prompt, image=scene_img, mask_image=mask).images[0]
        # Display the output image

        return generated_image

    except Exception as e:
        print(f"Error: {e}")
        return None

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://63304a621e0b91c95d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


**Setting up the UI**

In [ ]:
iface = gr.Interface(
    fn=generate_combined_image,
    inputs=[
        gr.Image(type="pil", label="Scene Image"),
        gr.Image(type="pil", label="Object Image")
    ],
    outputs=gr.Image(type="pil", label="Generated Image"),
    title="Inpainting with Stable Diffusion",
    description="Upload a scene image and an object image to generate a new image with the object placed in the scene."
)

# Launch the Gradio app
iface.launch()